# Charts

In [ ]:
import numpy as np
import pandas as pd
from siuba import *
from calitp import *
import intake
import data_prep
import _utils
import shared_utils
import altair as alt
import altair_saver
from shared_utils import altair_utils 
pd.options.display.max_rows = 100
pd.options.display.float_format = "{:.0f}".format
pd.options.display.float_format = '{:,}'.format

In [ ]:
#Natalie's cleaned df 
agg_parquet = pd.read_parquet('5311_agg.parquet')
agg_parquet.head(1)

In [ ]:
#Original DF
BC_GTFS_NTD2 = data_prep.merged_dataframe()
BC_GTFS_NTD2.head(1)

In [ ]:
#Grabbing only one row for each agency with original df
Aggregate1 = BC_GTFS_NTD2.groupby(['organization_name','reporter_type','GTFS', 'fleet_size', 'Is_Agency_In_BC_Only_1_means_Yes']).agg({'total_vehicles':'max',
       'average_age_of_fleet__in_years_':'max','doors_sum':'max','_0_9':'max', '_10_12':'max', '_13_15':'max', '_16_20':'max', '_21_25':'max',
       '_26_30':'max', '_31_60':'max', '_60plus':'max',})


In [ ]:
#How many orgs are static ok? 
Aggregate2['gtfs_static_status'].value_counts()

In [ ]:
Aggregate2['gtfs_realtime_status'].value_counts()

#  GTFS Status Charts
* One for vehicles over 9
* One for vehicles over 15

In [ ]:
vehicles_gtfs = _utils.aggregation_one(Aggregate1, 'GTFS')
vehicles_gtfs

In [ ]:
_utils.basic_bar_chart(vehicles_gtfs,'GTFS','vehicles_older_than_9','GTFS') 

In [ ]:
_utils.basic_bar_chart(vehicles_gtfs,'GTFS','vehicles_percent_older_than_9','GTFS') 

In [ ]:
_utils.basic_bar_chart(vehicles_gtfs,'GTFS','vehicles_older_than_15', 'GTFS') 

# Fleet size 

In [ ]:
fleet = _utils.aggregation_one(agg_parquet,'fleet_size') 
fleet

In [ ]:
# don't need no info
fleet =  fleet.loc[fleet['fleet_size'] != 'No Info'] 

In [ ]:
_utils.basic_bar_chart(fleet,'fleet_size','vehicles_older_than_9', 'fleet_size') 

### Vehicles older than 15
![pic](./chart_outputs/bar_fleet_size_by_vehicles_older_than_15.png)

In [ ]:
_utils.basic_bar_chart(fleet,'fleet_size','vehicles_older_than_15', 'fleet_size') 

In [ ]:
Orgs = agg_parquet.groupby(['gtfs_static_status']).agg({'organization_name':'nunique'})
Orgs = Orgs.reset_index()
Orgs = Orgs.rename(columns = {'organization_name': 'Organizations'}) 
_utils.basic_bar_chart(Orgs, 'gtfs_static_status','Organizations', 'gtfs_static_status') 


### Fleet Size by agencies

In [ ]:
Fleet = agg_parquet.groupby(['fleet_size']).agg({'organization_name':'nunique'}).reset_index()
Fleet

## Looking at reporter type & GTFS
![testing](./chart_outputs/bar_GTFS_by_count_of_agencies.png)

![testing](./chart_outputs/bar_reporter_type_by_Count of Agencies.png)

In [ ]:
agg_parquet.head(1)

In [ ]:
Reporter_type_agg = agg_parquet.groupby(['reporter_type']).agg({'organization_name':'nunique'})
Reporter_type_agg = Reporter_type_agg.reset_index()
Reporter_type_agg = Reporter_type_agg.rename(columns = {'organization_name':'Count_of_Agencies'})
_utils.basic_bar_chart(Reporter_type_agg,'reporter_type','Count_of_Agencies', 'reporter_type') 

In [ ]:
fleet_size_avg_age = agg_parquet.groupby(['fleet_size']).agg({'average_age_of_fleet__in_years_':'mean'})
fleet_size_avg_age

In [ ]:
Aggregate1.columns

In [ ]:
age = agg_parquet
age  =  age.loc[age['fleet_size'] != 'No Info'] 
age = age.groupby(['gtfs_realtime_status','fleet_size']).agg({'_0_9':'sum', 'vehicles_older_than_9':'sum','average_age_of_fleet__in_years_':'mean'})
age

## Looking at agencies that spent the most..

In [ ]:
Most_Money = agg_parquet.groupby(['organization_name','gtfs_realtime_status']).agg({'allocationamount':'sum'})

In [ ]:
Most_Money.sort_values('allocationamount')

In [ ]:
Top_ten = Most_Money.sort_values('allocationamount').tail(10)
Top_ten =Top_ten.rename(columns = {'allocationamount': '5311_funds_received'})
Top_ten = Top_ten.reset_index()

In [ ]:

Top_ten_agencies = Top_ten['organization_name'].tolist()
Top_ten_agencies

In [ ]:
Year_look = BC_GTFS_NTD2.groupby(['grant_fiscal_year','organization_name']).agg({'allocationamount':'sum'})
Year_look = Year_look.reset_index()

Year_look = Year_look[Year_look.organization_name.isin(Top_ten_agencies)]
Year_look.groupby(['organization_name','grant_fiscal_year']).agg({'allocationamount':'sum'})

In [ ]:
_utils.basic_bar_chart(Top_ten,'organization_name','5311_funds_received', 'organization_name') 

In [ ]:
bc_funds = BC_GTFS_NTD2.groupby(['funding_program']).agg({'organization_name':'nunique','allocationamount':'sum'})
bc_funds = bc_funds.rename(columns = {'organization_name':'Count_of_Organizations','allocationamount':'total_sum'}).reset_index()
bc_funds.sort_values(by =['total_sum'])

_utils.basic_bar_chart(bc_funds,'funding_program','total_sum', 'funding_program') 

In [ ]:
bc_funds

In [ ]:
129/164

In [ ]:
BC_GTFS_NTD2.columns

In [ ]:
project_money = BC_GTFS_NTD2.groupby(['description']).agg({'upin':'nunique','allocationamount':'sum'}).rename(columns = {'allocationamount': '5311_funds_received'})

In [ ]:

project_money = project_money.sort_values('5311_funds_received').tail(10).reset_index()

In [ ]:
_utils.basic_bar_chart(project_money,'description','5311_funds_received', 'description') 